In [1]:
import requests
import string
from datetime import datetime
from datetime import timedelta

from lxml import etree
from lxml import html

from pandas import DataFrame as df

In [2]:
currentDate = datetime.now()
yyyy = currentDate.year
mm = currentDate.month
dd = currentDate.day

dayback =timedelta(days=1, hours=22)
dateToExtact = currentDate - dayback

ext_yyyy = dateToExtact.year
ext_mm = dateToExtact.month
ext_dd = dateToExtact.day

queryParam = {'fromDate': ext_yyyy.__str__() + '-' + ext_mm.__str__() + '-' + ext_dd.__str__()}

queryParam

{'fromDate': '2016-1-6'}

In [4]:

proxies = {'http': 'proxy.wellsfargo.com:8080', 'https': 'proxy.wellsfargo.com:8080'}
parser = etree.HTMLParser()

# r = requests.get('http://www.hongkongairport.com/flightinfo/chi/real_arrinfo.do', params=queryParam, proxies=proxies)
r = requests.get('http://www.hongkongairport.com/flightinfo/chi/real_arrinfo.do', params=queryParam)

# r_dep = requests.get('http://www.hongkongairport.com/flightinfo/chi/real_depinfo.do', params=queryParam, proxies=proxies)
r_dep = requests.get('http://www.hongkongairport.com/flightinfo/chi/real_depinfo.do', params=queryParam)

r_dep.url

u'http://www.hongkongairport.com/flightinfo/chi/real_depinfo.do?fromDate=2016-1-6'

In [5]:
flightTable = []

tree = etree.fromstring(r.content, parser=parser)
trList =tree.xpath('//table[@id=\'list\']/tr[not(@mr)]')

for tr in trList:
    if 'date' in tr.attrib:
        dateStr = tr.attrib['date'].split(',')[1]
        yyyy=int(dateStr.split('-')[0])
        mm=int(dateStr.split('-')[1])
        dd=int(dateStr.split('-')[2])
    
    scheduledTimeStr = tr.getchildren()[0].text
    schHH = int(scheduledTimeStr.split(':')[0])
    schMM = int(scheduledTimeStr.split(':')[1])
    
    scheduledTime = datetime(yyyy, mm, dd, schHH, schMM)
    
    flightCode = tr.getchildren()[1].text
    
    city = tr.getchildren()[2].text
    
    airline = ''
    airlineRaw = tr.getchildren()[3].text
    for airlineStr in airlineRaw.split(','):
        if airlineStr.find('wm') == -1:
            airline = airline + ',' + airlineStr
    airline = airline[1:]
    
    gate = tr.getchildren()[4].text
    
    arriveTimeStr = tr.getchildren()[5].text
    if arriveTimeStr.find(':') == -1 :
        arriveTime = None
    elif arriveTimeStr.find('(') == -1 :
        timeStr = arriveTimeStr.split(' ')[1]
        arriveTime = datetime(yyyy, mm, dd, int(timeStr.split(':')[0]), int(timeStr.split(':')[1]))
    else :
        dateStr = arriveTimeStr.split('(')[1].split(')')[0]
        timeStr = arriveTimeStr.split(' ')[1]
        
        yyyyActual = int(dateStr.split('/')[2])
        mmActual = int(dateStr.split('/')[1])
        ddActual = int(dateStr.split('/')[0])
        
        hourActual = int(timeStr.split(':')[0])
        minActual = int(timeStr.split(':')[1])
        
        arriveTime = datetime(yyyyActual, mmActual, ddActual, hourActual, minActual)
        
    flightEntry = {'scheduledTime': scheduledTime, 
                  'flightCode': flightCode,
                  'city': city,
                  'airline': airline,
                  'gate': gate,
                  'arriveTime': arriveTime}
    
    flightTable.append(flightEntry)
    
flightData = df(flightTable)

flightData.head()


,airline,arriveTime,city,flightCode,gate,scheduledTime
0,"港龍航空公司,國泰航空公司",2016-01-06 00:37:00,河內,"KA 294,CX 5294",B,2016-01-05 22:20:00
1,"國泰航空公司,芬蘭航空,曼谷航空公司,美國航空",2016-01-06 01:08:00,曼谷,"CX 702,AY 5858,PG 4567,AA 8894",B,2016-01-05 22:50:00
2,韓亞航空,2016-01-06 00:19:00,首爾/仁川,OZ 749,B,2016-01-05 23:00:00
3,"國泰航空公司,日本航空",2016-01-06 00:49:00,札幌,"CX 583,JL 7065",B,2016-01-05 23:40:00
4,"埃塞俄比亞航空,韓亞航空",2016-01-06 00:10:00,首爾/仁川,"ET 609,OZ 9767",B,2016-01-05 23:45:00


In [6]:
flightData[flightData.arriveTime.isnull()]

,airline,arriveTime,city,flightCode,gate,scheduledTime
97,"香港航空,海南航空",NaT,福州,"HX 174,HU 8214",,2016-01-06 10:10:00
153,"國泰航空公司,港龍航空公司",NaT,台北,"CX 469,KA 5469",,2016-01-06 12:05:00
162,"國泰航空公司,港龍航空公司",NaT,上海/浦東,"CX 365,KA 5365",,2016-01-06 12:30:00
167,"中國東方航空,香港航空,上海航空公司",NaT,上海/浦東,"MU 701,HX 1701,FM 701",,2016-01-06 12:40:00
178,"港龍航空公司,國泰航空公司",NaT,高雄,"KA 433,CX 5433",,2016-01-06 12:55:00
345,"香港航空,阿提哈德航空公司",NaT,曼谷,"HX 776,EY 6164",,2016-01-06 18:55:00
364,"香港航空,上海航空公司,中國東方航空",NaT,上海/浦東,"HX 241,FM 8241,MU 8991",,2016-01-06 19:40:00


In [7]:
flightTable_dep = []

tree_dep = etree.fromstring(r_dep.content, parser=parser)
trList_dep =tree_dep.xpath('//table[@id=\'list\']/tr[not(@mr)]')


for tr in trList_dep:
    if 'date' in tr.attrib:
        dateStr = tr.attrib['date'].split(',')[1]
        yyyy=int(dateStr.split('-')[0])
        mm=int(dateStr.split('-')[1])
        dd=int(dateStr.split('-')[2])
        
    scheduledTimeStr = tr.getchildren()[0].text
    schHH = int(scheduledTimeStr.split(':')[0])
    schMM = int(scheduledTimeStr.split(':')[1])
    
    scheduledTime = datetime(yyyy, mm, dd, schHH, schMM)
    
    flightCode = tr.getchildren()[1].text
    city = tr.getchildren()[2].text
    terminal = tr.getchildren()[3].text
    checkIn = tr.getchildren()[4].text
    try:
        gate = int(tr.getchildren()[5].text)
    except ValueError:
        gate = None
    
    airline = ''
    airlineRaw = tr.getchildren()[6].text
    for airlineStr in airlineRaw.split(','):
        if airlineStr.find('wm') == -1:
            airline = airline + ',' + airlineStr
    airline = airline[1:]
    
    departTimeStr = tr.getchildren()[7].text
    if departTimeStr.find(':') == -1 :
        departTime = None
    elif departTimeStr.find('(') == -1 :
        timeStr = departTimeStr.split(' ')[1]
        departTime = datetime(yyyy, mm, dd, int(timeStr.split(':')[0]), int(timeStr.split(':')[1]))
    else :
        dateStr = departTimeStr.split('(')[1].split(')')[0]
        timeStr = departTimeStr.split(' ')[1]
        
        yyyyActual = int(dateStr.split('/')[2])
        mmActual = int(dateStr.split('/')[1])
        ddActual = int(dateStr.split('/')[0])
        hourActual = int(timeStr.split(':')[0])
        minActual = int(timeStr.split(':')[1])
        departTime = datetime(yyyyActual, mmActual, ddActual, hourActual, minActual)
            
    flightEntry = {'scheduledTime': scheduledTime, 
                   'flightCode': flightCode,
                   'city': city,
                   'terminal': terminal,
                   'checkIn': checkIn,
                   'airline': airline,
                   'gate': gate,
                   'departTime': departTime}
    
    flightTable_dep.append(flightEntry)


flightData_dep = df(flightTable_dep)

flightData_dep.head()

,airline,checkIn,city,departTime,flightCode,gate,scheduledTime,terminal
0,"國泰航空公司,英國航空公司,日本航空",C,泗水,2016-01-06 03:15:00,"CX 781D,BA 4571,JL 7921",25,2016-01-05 15:35:00,T1
1,英國航空公司,J,倫敦/希斯路,2016-01-06 00:13:00,BA 028,17,2016-01-05 23:45:00,T1
2,"南非航空公司,全日空,加拿大航空,韓亞航空",F,約翰尼斯堡,2016-01-06 00:07:00,"SA 287,NH 6665,AC 6874,OZ 6593",35,2016-01-05 23:50:00,T1
3,國泰航空公司,C,倫敦/希斯路,2016-01-06 00:33:00,CX 251,1,2016-01-05 23:55:00,T1
4,維珍航空,E,倫敦/希斯路,2016-01-06 00:16:00,VS 207,27,2016-01-05 23:55:00,T1


In [8]:
flightData_dep[flightData_dep.departTime.isnull()]

,airline,checkIn,city,departTime,flightCode,gate,scheduledTime,terminal
84,"港龍航空公司,國泰航空公司",H,高雄,NaT,"KA 432,CX 5432",NaN,2016-01-06 08:55:00,T1
131,"中國國際航空公司,國泰航空公司,港龍航空公司",J,北京,NaT,"CA 108,CX 6108,KA 1108",NaN,2016-01-06 10:30:00,T1
192,"香港航空,阿提哈德航空公司",K,曼谷,NaT,"HX 775,EY 6169",NaN,2016-01-06 12:30:00,T1
204,"香港航空,上海航空公司,中國東方航空",K,上海/浦東,NaT,"HX 246,FM 8246,MU 8989",NaN,2016-01-06 13:00:00,T1
221,"中國東方航空,上海航空公司,香港航空",J,上海/浦東,NaT,"MU 702,FM 702,HX 1702",NaN,2016-01-06 13:40:00,T1


In [26]:
matchFlightCode = flightData_dep['flightCode'].apply(lambda x: 'FM 8246' in x)
flightData_dep[matchFlightCode]

,airline,checkIn,city,departTime,flightCode,gate,scheduledTime,terminal
204,"香港航空,上海航空公司,中國東方航空",K,上海/浦東,NaT,"HX 246,FM 8246,MU 8989",NaN,2016-01-06 13:00:00,T1
